In [1]:
import os
from glob import glob
import random
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2 as cv
from statannot import add_stat_annotation

Button

In [50]:
test_type = 'elec'
base_path = f'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision'

num_class = 16
if test_type == 'opt':
  sel_ppl = list(range(300, 309)) + list(range(400, 408)) + [611]
elif test_type == 'elec': # check
  sel_ppl = [499, 500] + list(range(502, 509)) + list(range(601, 607)) + list(range(608, 611)) # 18 subjects

quesdata_files = glob(f'{base_path}\\data\\Question_Banks_AI_Hub_final\\*classes\\set0\\*')
humsub_path = glob(f'{base_path}\\data\\210827_ANNA_Removing_uncontaminated_data.csv')[0]
testdata_path = glob(f'{base_path}\\data\\sample_for_dev_test_{test_type}')[0]
fig_path = glob(f'{base_path}\\figures')[0]

Machine Data

In [51]:
quesdata_files

['C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\seed1',
 'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\seed100',
 'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\seed2',
 'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\seed22',
 'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\seed42',
 'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\seed50',
 'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\seed55',
 'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\seed7',
 '

In [47]:
model_type1_list = [''] #['PCA', 'PCA', '', '']
model_type2_list = ['CNN_SVC'] #['SVC', 'LR', 'CNN_LR', 'CNN_SVC']
seed_list = [22, 77, 2, 100, 81, 42, 7, 1, 55, 50] 
pix_order_list = ['16PIX', '24PIX', '32PIX', '64PIX', '128PIX']
gs_order_list = ['2GS', '4GS', '6GS', '8GS', '16GS']
class_list = [16, 2]

df = pd.read_csv(humsub_path)
l = list(range(df.shape[0]))
n = 16
random.seed(22)
set_1 = random.sample(l, n)
sets = [set_1]


for (model_type1, model_type2) in zip(model_type1_list, model_type2_list):
    for c_num in class_list:
        model_type = model_type1 + model_type2
        input_folder = [df.iloc[i, 0] for i in sets[0]] 
        assert len(input_folder) == 16
        com_obj = itertools.combinations(input_folder, 16)
        com_list = list(com_obj)

        mac_df = pd.DataFrame()
        for i in range(len(quesdata_files)):
            data_path = quesdata_files[i]
            seed = int(os.path.basename(data_path).split('d')[-1])
            if c_num == int(data_path.split('\\')[-3].split('c')[0]):
                for n in range(len(os.listdir(data_path))):
                    print(seed, n)

                    preprocessed_data_path =  os.path.join(data_path, f'comb{n}') 
                    high_analysis_path = os.path.join(preprocessed_data_path, f'High_Analysis_{test_type}')
                    
                    if os.path.exists(os.path.join(high_analysis_path, f'High_Level_Data_Analysis_{model_type1}_{model_type2}.csv')):
                        add_high_df = pd.read_csv(os.path.join(high_analysis_path, f'High_Level_Data_Analysis_{model_type1}_{model_type2}.csv'))
                        add_high_df['Hit Rate'] = add_high_df['correctness'].replace(['correct', 'wrong'], [1, 0]) 
                        add_high_df = add_high_df[['file_name', 'actual_person', 'Hit Rate']]
                        add_high_df['actual_person'] = add_high_df['actual_person'].astype(int)
                        
                        img_list, hyperpar_list, par_list = [], [], []
                        for i in range(add_high_df.shape[0]):
                            file_name = add_high_df['file_name'][i].split('.')[0]
                            pix, gs, par = file_name.split('_')
                            per = add_high_df['actual_person'][i]
                            img = str(per) + '_' + str(pix) + '_' + str(gs) + '_' + str(par) + '.jpg'
                            img_list.append(img)
                            hyperpar_list.append(str(pix) + '_' + str(gs))
                            par_list.append(str(par))
                        add_high_df['img'] = img_list
                        add_high_df['hyperpar'] = hyperpar_list
                        add_high_df['par'] = par_list
                        
                        add_high_df = add_high_df[['hyperpar', 'Hit Rate']]
                    else:
                        break
                
                add_high_df['Seed'] = [seed] * add_high_df.shape[0]
                add_high_df['Class'] = [c_num] * add_high_df.shape[0]
                
                mac_df = pd.concat([mac_df, add_high_df], axis=0)

# mac_df = mac_df.groupby(['hyperpar', 'Class']).mean()['Hit Rate'].to_frame().reset_index()

# pix_lst = []
# for hyperpar in mac_df['hyperpar']:
#     pix, _ = hyperpar.split('_')
#     pix_lst.append(pix)
# mac_df['pix'] = pix_lst

mac_df

1 0
100 0
2 0
22 0
42 0
50 0
55 0
7 0
77 0
81 0
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
100 0
100 1
100 2
100 3
100 4
100 5
100 6
100 7
100 8
100 9
100 10
100 11
100 12
100 13
100 14
100 15
100 16
100 17
100 18
100 19
100 20
100 21
100 22
100 23
100 24
100 25
100 26
100 27
100 28
100 29
100 30
100 31
100 32
100 33
100 34
100 35
100 36
100 37
100 38
100 39
100 40
100 41
100 42
100 43
100 44
100 45
100 46
100 47
100 48
100 49
10

,hyperpar,Hit Rate,Seed,Class
0,128PIX_16GS,1,1,2
1,128PIX_16GS,1,1,2
2,128PIX_16GS,1,1,2
3,128PIX_16GS,1,1,2
4,128PIX_16GS,1,1,2
...,...,...,...,...
445,64PIX_8GS,1,81,2
446,64PIX_8GS,1,81,2
447,64PIX_8GS,1,81,2
448,64PIX_8GS,1,81,2


In [49]:
mac_df[mac_df['Class']==16]

,hyperpar,Hit Rate,Seed,Class


In [41]:
mac_df.to_csv(os.path.join(fig_path, f'{class_list[0]}vs{class_list[1]}classes_{test_type}.csv'))

In [45]:
add_high_df

,hyperpar,Hit Rate,Seed,Class
0,128PIX_16GS,1,81,16
1,128PIX_16GS,1,81,16
2,128PIX_16GS,1,81,16
3,128PIX_16GS,1,81,16
4,128PIX_16GS,1,81,16
...,...,...,...,...
3595,64PIX_8GS,1,81,16
3596,64PIX_8GS,1,81,16
3597,64PIX_8GS,1,81,16
3598,64PIX_8GS,1,81,16
